<a href="https://colab.research.google.com/github/gkianfar/TIHM-Dataset-Visualization/blob/main/tihm_featureExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Clone Github repo and set working directory path**

In [1]:
!git clone https://github.com/gkianfar/TIHM-Dataset-Visualization

Cloning into 'TIHM-Dataset-Visualization'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 66 (delta 30), reused 13 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (66/66), 14.10 MiB | 5.55 MiB/s, done.
Resolving deltas: 100% (30/30), done.


**Load packages**

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib import cm
import matplotlib.patches as mpatches
from collections.abc import Iterable
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')
from matplotlib import cm
#from utils import *

%matplotlib inline
## Set colour palette
ibm_colorblind = ['#648FFF', '#FE6100', '#DC267F', '#785EF0', '#FFB000','#48A9A6']
sns.set_palette(ibm_colorblind)

**Functions**

In [5]:


def correct_col_type(df,col):
    raw_type = str(type(df[col].dtype)).split('.')[-1].split('\'')[0].lower()
    #print(col,raw_type)
    if 'object' in raw_type:
        if 'date' in col or 'timestamp' in col or 'datetime' in col:
            return pd.to_datetime(df[col])
        else:
            return df[col].astype('category')
    else:
        return df[col]


def gen_date_col(df, tcol):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['date'] = df[tcol].dt.date
    return df





**Set Data and Fig folders paths**

In [6]:
## Please change the path with the path of your dataset
%cd /content/TIHM-Dataset-Visualization
DPATH = './Data'
SAVE_PATH = './Figs/'
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)



/content/TIHM-Dataset-Visualization


In [8]:
delta = 'sec' # minute
interval = 6 #hours
if delta == 'sec':
  interval = interval*3600
elif delta == 'min':
  interval = interval*60





**Import Files**

In [9]:
### Labels
f = 'Labels.csv'
fpth = os.path.join(DPATH,f)
labels_df = pd.read_csv(fpth)
for col in labels_df.columns:
    labels_df[col] = correct_col_type(labels_df,col)
if 'date' in labels_df.columns:
    labels_df = labels_df.rename(columns={'date':'timestamp'})
if 'type' in labels_df.columns:
    labels_df = labels_df.rename(columns={'type':'label'})
labels_df = gen_date_col(labels_df,tcol='timestamp')

labels_patients = np.unique(labels_df['patient_id'])

Agitation_labels = labels_df.loc[labels_df['label'] == 'Agitation']
Agitation_labels

,patient_id,timestamp,label,date
2,16f4b,2019-04-11 12:00:22,Agitation,2019-04-11
4,16f4b,2019-04-14 12:00:07,Agitation,2019-04-14
5,16f4b,2019-04-15 18:00:24,Agitation,2019-04-15
6,16f4b,2019-04-16 18:00:38,Agitation,2019-04-16
11,16f4b,2019-04-21 12:00:55,Agitation,2019-04-21
...,...,...,...,...
587,0d5ef,2019-06-28 18:02:11,Agitation,2019-06-28
588,d7a46,2019-06-28 18:02:21,Agitation,2019-06-28
598,6b29b,2019-06-29 12:00:27,Agitation,2019-06-29
599,95899,2019-06-29 12:01:20,Agitation,2019-06-29


In [10]:
### Activity

f = 'Activity.csv'
fpth = os.path.join(DPATH,f)
df = pd.read_csv(fpth)
for col in df.columns:
    df[col] = correct_col_type(df,col)
if 'date' in df.columns:
    activity_df = df.rename(columns={'date':'timestamp'})
    activity_df['date'] = pd.to_datetime(activity_df['timestamp'].dt.date)


activity_patients = np.unique(activity_df['patient_id'])

In [11]:
### Physiology
f = 'Physiology.csv'
fpth = os.path.join(DPATH,f)
df = pd.read_csv(fpth)
for col in df.columns:
    df[col] = correct_col_type(df,col)
if 'date' in df.columns:
    physiology_df = df.rename(columns={'date':'timestamp'})
    physiology_df['date'] = pd.to_datetime(physiology_df['timestamp'].dt.date)
#physiology_df = gen_date_col(df,tcol='timestamp')

physiology_patients = np.unique(physiology_df['patient_id'])

In [12]:
### Physiology
f = 'Sleep.csv'
fpth = os.path.join(DPATH,f)
df = pd.read_csv(fpth)
for col in df.columns:
    df[col] = correct_col_type(df,col)
if 'date' in df.columns:
    sleep_df = df.rename(columns={'date':'timestamp'})
    sleep_df['date'] = pd.to_datetime(sleep_df['timestamp'].dt.date)
#physiology_df = gen_date_col(df,tcol='timestamp')

sleep_patients = np.unique(sleep_df['patient_id'])

In [18]:
interval = 6 # hours to filter activities
agitation_df = labels_df[labels_df['label']=='Agitation'].copy()
agitation_location = agitation_df[['patient_id', 'timestamp']]
agitation_patientID = agitation_df['patient_id'].tolist()
agitation_endTime = agitation_df['timestamp'].tolist()
### 6 hours earlier
agitation_startTime = pd.to_datetime(agitation_endTime) - pd.Timedelta(hours=interval)
agitation_startTime = agitation_startTime.tolist()

### patients that have agitation and the number of samples that each has
agitation_patients = np.unique(agitation_location['patient_id'])
each_patient_agitation_sample = agitation_df['patient_id'].value_counts()


# Create a list to hold filtered activities per input label samples for agitation detection
agitation_activity = []

# Iterate through each patient_id, start_time, and end_time and filter related activities between agitation_startTime and agitation_endTime
for patient_id, start_time, end_time in zip(agitation_patientID, agitation_startTime, agitation_endTime):
    # Filter the DataFrame for the current patient and time range
    filtered_df = activity_df[(activity_df['patient_id'] == patient_id) &
                            (activity_df['timestamp'] >= start_time) &
                            (activity_df['timestamp'] <= end_time)]

    # Store the results in the list
    filtered_activity = filtered_df['location_name'].tolist()
    agitation_activity.append(filtered_activity)


In [67]:
# Mapping function for all categories in Location_name column

activity_locations_unique = activity_df['location_name'].unique()
activity_locations_map = {location: index for index, location in enumerate(activity_locations_unique)}

activity_change_count = []
activity_occurance_count = []
for filtered_activity in agitation_activity:

  dummy_filtered_activity = pd.Series(filtered_activity)
  activity_occurance_count.append(dummy_filtered_activity.value_counts())

  dummy_filtered_activity = dummy_filtered_activity.map(activity_locations_map)
  change_count = dummy_filtered_activity.diff().value_counts()

  if any(change_count.index == 0):
    unchanged_count = change_count[change_count.index == 0].values[0]
  else :
    unchanged_count = 0

  # Counting number of changes between locations by substracting the total number of unchanged locations from the max possible changes
  activity_change_count.append((dummy_filtered_activity.shape[0]-1)-unchanged_count)



In [68]:
activity_change_count

[187,
 110,
 133,
 278,
 154,
 165,
 196,
 191,
 260,
 222,
 160,
 165,
 146,
 182,
 246,
 117,
 196,
 48,
 200,
 165,
 160,
 140,
 145,
 2,
 45,
 120,
 192,
 146,
 220,
 188,
 164,
 249,
 173,
 208,
 50,
 132,
 173,
 147,
 91,
 132,
 153,
 48,
 174,
 149,
 179,
 284,
 168,
 145,
 160,
 223,
 37,
 126,
 171,
 253,
 161,
 198,
 205,
 196,
 121,
 137,
 155,
 147,
 271,
 169,
 122,
 134,
 216,
 213,
 174,
 143,
 143,
 103,
 115,
 154,
 149,
 59,
 158,
 146,
 140,
 23,
 107,
 119,
 166,
 240,
 129,
 257,
 214,
 199,
 227,
 216,
 149,
 171,
 133,
 65,
 157,
 170,
 190,
 198,
 232,
 145,
 -1,
 118,
 105,
 262,
 185,
 119,
 206,
 148,
 104,
 106,
 204,
 103,
 169,
 228,
 156,
 170,
 151,
 192,
 225,
 217,
 102,
 183,
 197,
 46,
 215,
 178,
 181,
 198,
 161,
 113,
 203,
 218,
 137,
 190,
 256]